In [1]:
# IMPORTS & DEFINITIONS

import csv, sys
import datetime
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np; np.set_printoptions(threshold=sys.maxsize)
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
import math
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures

DUD_VALUE= 0 # change from 0 to something like 123 for debugging
EMPTY_DATA_DAY_VAL= 123456789
TOTAL_ROWS= 9999999999
INPUT_ROWS_LIMIT= TOTAL_ROWS # 500000
FILENAME= 'dublinbikes_2020_Q1.csv'
MAX_STATIONS= 117
SECS_IN_5MIN= 300
DATAPOINT_EVERYX_MIN= 5
DATAPOINTS_PER_DAY= 288
DAYS_OF_WEEK= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'] # yes, I consider Monday to be the '0'/start of the week
STARTING_DATE= 0 # aka Monday. Because the 27th of Jan 2020 is a Monday
MISSING_STATIONS= [116, 70, 60, 46, 35, 20, 14, 1, 0]
NUM_STATIONS= MAX_STATIONS - len(MISSING_STATIONS)
SUBSTANDARD_DAYS= [] # [50, 49]
TOTAL_DAYS= 66 # from 27 / 1 / 2020 to (and including) 1 / 4 / 2020
HOURS= 24
EPOCH= datetime.datetime(2020, 1, 27, 0, 0)
TOTAL_TIME_DATAPOINTS= int((datetime.datetime(2020,4,2,0,0) - EPOCH).total_seconds() / SECS_IN_5MIN)
K= 5
STEP_SIZE= 0.02185 # just the magic number that leads to 288 values being generated
R= 0.5
MAX_HINDSIGHT= 60 # minutes
DAYS_PER_WEEKDAY= 5

class DataDay: # ideally this would be nested in the Station class
    def __init__(self, index):
        self.index= index
        self.substandard_day= False
        if index in SUBSTANDARD_DAYS:
            self.substandard_day= True
        self.times_populated= 0
        self.day_of_week= ((STARTING_DATE + index) % len(DAYS_OF_WEEK))
        
        self.daily_epoch_time= np.full(DATAPOINTS_PER_DAY, EMPTY_DATA_DAY_VAL, dtype=np.int)
        self.epoch_time= np.full(DATAPOINTS_PER_DAY, EMPTY_DATA_DAY_VAL, dtype=np.int)
        self.bikes= np.full(DATAPOINTS_PER_DAY, EMPTY_DATA_DAY_VAL, dtype=np.int)
        self.percent_bikes= np.full(DATAPOINTS_PER_DAY, float(EMPTY_DATA_DAY_VAL), dtype=np.float)

    def populate(self, daily_epoch_time, epoch_time, bikes, percent_bikes):
        if self.substandard_day == False:
            self.daily_epoch_time[daily_epoch_time]= daily_epoch_time
            self.epoch_time[daily_epoch_time]= epoch_time
            self.bikes[daily_epoch_time]= bikes
            self.percent_bikes[daily_epoch_time]= percent_bikes
            self.times_populated+= 1

class Station:
    def __init__(self, index):
        self.index= index
        self.name= DUD_VALUE
        self.bike_capacity= DUD_VALUE
        self.address= DUD_VALUE
        self.latitude= DUD_VALUE
        self.longitude= DUD_VALUE
        self.data_days= [DataDay(i) for i in range(0, TOTAL_DAYS)]
    
    def populate_consts(self, name, bike_capacity, address, latitude, longitude):
        self.name= name
        self.bike_capacity= bike_capacity
        self.address= address
        self.latitude= latitude
        self.longitude= longitude

def get_station_id(name):
    try:
        index= [x.name for x in stations].index(name)
    except ValueError:
        index= -1
    return index

In [2]:
# DATA STRUCTURING

total_capacity= 0 # not in use currently
index= []; daily_epoch_time= []; epoch_time= []; percent_bikes= [];
stations= [Station(i) for i in range(0, MAX_STATIONS)]
indices_to_populate= list(range(0, MAX_STATIONS))
for index in MISSING_STATIONS:
    indices_to_populate.remove(index)

with open(FILENAME, newline='') as f:
    reader = csv.reader(f); next(reader) # skip data header
    current_index= 0
    try:
        while len(indices_to_populate) != 0:
            row= next(reader)
            if int(row[0]) == current_index: # this clause is just for performance
                continue
            current_index= int(row[0])
            if current_index in indices_to_populate:
                stations[current_index].populate_consts(row[3], row[4], row[8], row[9], row[10])
                indices_to_populate.remove(current_index)
                total_capacity+= int(row[4])
        
        f.seek(0)
        reader= csv.reader(f); row= next(reader) # skip data header
        for row_i, row in enumerate(reader):
            if row_i >= INPUT_ROWS_LIMIT:
                break
            if int((datetime.datetime(int(row[1][0:4]), int(row[1][5:7]), int(row[1][8:10]), int(row[1][11: 13]), int(row[1][14: 16])) - EPOCH).total_seconds()) < 0:
                continue
            try:
                epoch_time= int((datetime.datetime(int(row[1][0:4]), int(row[1][5:7]), int(row[1][8:10]), int(row[1][11: 13]), int(row[1][14: 16])) - EPOCH).total_seconds() / SECS_IN_5MIN)
                stations[int(row[0])].data_days[int(epoch_time / DATAPOINTS_PER_DAY)].populate( \
                    int((datetime.datetime(int(row[1][0:4]), int(row[1][5:7]), int(row[1][8:10]), int(row[1][11: 13]), int(row[1][14: 16])) - datetime.datetime(int(row[1][0:4]), int(row[1][5:7]), int(row[1][8:10]), 0, 0)).total_seconds() / (SECS_IN_5MIN)), \
                    epoch_time, \
                    int(row[6]), \
                    float("{:.3f}".format(float(row[6]) / float(row[4]))))
            except IndexError as e:
                print("\nTRIED: ", epoch_time, ' / ', DATAPOINTS_PER_DAY, ' = ', int(epoch_time / DATAPOINTS_PER_DAY))
                print(row[1])
    except csv.Error as e:
        sys.exit('file {}, line {}: {}'.format(filename, reader.line_num, e))


TRIED:  746  /  288  =  2
2020-01-29 14:10:02

TRIED:  747  /  288  =  2
2020-01-29 14:15:02

TRIED:  748  /  288  =  2
2020-01-29 14:20:02

TRIED:  749  /  288  =  2
2020-01-29 14:25:02

TRIED:  750  /  288  =  2
2020-01-29 14:30:02

TRIED:  751  /  288  =  2
2020-01-29 14:35:02

TRIED:  752  /  288  =  2
2020-01-29 14:40:02

TRIED:  753  /  288  =  2
2020-01-29 14:45:02

TRIED:  754  /  288  =  2
2020-01-29 14:50:02

TRIED:  755  /  288  =  2
2020-01-29 14:55:02

TRIED:  756  /  288  =  2
2020-01-29 15:00:02

TRIED:  757  /  288  =  2
2020-01-29 15:05:02

TRIED:  758  /  288  =  2
2020-01-29 15:10:02

TRIED:  759  /  288  =  2
2020-01-29 15:15:02

TRIED:  760  /  288  =  2
2020-01-29 15:20:02

TRIED:  761  /  288  =  2
2020-01-29 15:25:02

TRIED:  762  /  288  =  2
2020-01-29 15:30:02

TRIED:  763  /  288  =  2
2020-01-29 15:35:02

TRIED:  764  /  288  =  2
2020-01-29 15:40:02

TRIED:  765  /  288  =  2
2020-01-29 15:45:02

TRIED:  766  /  288  =  2
2020-01-29 15:50:02

TRIED:  767 

TRIED:  1080  /  288  =  3
2020-01-30 18:00:02

TRIED:  1081  /  288  =  3
2020-01-30 18:05:02

TRIED:  1082  /  288  =  3
2020-01-30 18:10:02

TRIED:  1083  /  288  =  3
2020-01-30 18:15:02

TRIED:  1084  /  288  =  3
2020-01-30 18:20:02

TRIED:  1085  /  288  =  3
2020-01-30 18:25:02

TRIED:  1086  /  288  =  3
2020-01-30 18:30:27

TRIED:  1087  /  288  =  3
2020-01-30 18:35:02

TRIED:  1088  /  288  =  3
2020-01-30 18:40:02

TRIED:  1089  /  288  =  3
2020-01-30 18:45:02

TRIED:  1090  /  288  =  3
2020-01-30 18:50:02

TRIED:  1091  /  288  =  3
2020-01-30 18:55:02

TRIED:  1092  /  288  =  3
2020-01-30 19:00:02

TRIED:  1093  /  288  =  3
2020-01-30 19:05:02

TRIED:  1094  /  288  =  3
2020-01-30 19:10:02

TRIED:  1095  /  288  =  3
2020-01-30 19:15:02

TRIED:  1096  /  288  =  3
2020-01-30 19:20:02

TRIED:  1097  /  288  =  3
2020-01-30 19:25:02

TRIED:  1098  /  288  =  3
2020-01-30 19:30:02

TRIED:  1099  /  288  =  3
2020-01-30 19:35:02

TRIED:  1100  /  288  =  3
2020-01-30 19

TRIED:  1373  /  288  =  4
2020-01-31 18:25:02

TRIED:  1374  /  288  =  4
2020-01-31 18:30:02

TRIED:  1375  /  288  =  4
2020-01-31 18:35:02

TRIED:  1376  /  288  =  4
2020-01-31 18:40:02

TRIED:  1377  /  288  =  4
2020-01-31 18:45:02

TRIED:  1378  /  288  =  4
2020-01-31 18:50:02

TRIED:  1379  /  288  =  4
2020-01-31 18:55:02

TRIED:  1380  /  288  =  4
2020-01-31 19:00:02

TRIED:  1381  /  288  =  4
2020-01-31 19:05:02

TRIED:  1382  /  288  =  4
2020-01-31 19:10:02

TRIED:  1383  /  288  =  4
2020-01-31 19:15:02

TRIED:  1384  /  288  =  4
2020-01-31 19:20:02

TRIED:  1385  /  288  =  4
2020-01-31 19:25:02

TRIED:  1386  /  288  =  4
2020-01-31 19:30:02

TRIED:  1387  /  288  =  4
2020-01-31 19:35:02

TRIED:  1388  /  288  =  4
2020-01-31 19:40:02

TRIED:  1389  /  288  =  4
2020-01-31 19:45:02

TRIED:  1390  /  288  =  4
2020-01-31 19:50:02

TRIED:  1391  /  288  =  4
2020-01-31 19:55:02

TRIED:  1392  /  288  =  4
2020-01-31 20:00:02

TRIED:  1393  /  288  =  4
2020-01-31 20

TRIED:  1679  /  288  =  5
2020-02-01 19:55:02

TRIED:  1680  /  288  =  5
2020-02-01 20:00:02

TRIED:  1681  /  288  =  5
2020-02-01 20:05:02

TRIED:  1682  /  288  =  5
2020-02-01 20:10:01

TRIED:  1683  /  288  =  5
2020-02-01 20:15:02

TRIED:  1684  /  288  =  5
2020-02-01 20:20:02

TRIED:  1685  /  288  =  5
2020-02-01 20:25:02

TRIED:  1686  /  288  =  5
2020-02-01 20:30:02

TRIED:  1687  /  288  =  5
2020-02-01 20:35:02

TRIED:  1688  /  288  =  5
2020-02-01 20:40:02

TRIED:  1689  /  288  =  5
2020-02-01 20:45:02

TRIED:  1690  /  288  =  5
2020-02-01 20:50:02

TRIED:  1691  /  288  =  5
2020-02-01 20:55:02

TRIED:  1692  /  288  =  5
2020-02-01 21:00:02

TRIED:  1693  /  288  =  5
2020-02-01 21:05:02

TRIED:  1694  /  288  =  5
2020-02-01 21:10:02

TRIED:  1695  /  288  =  5
2020-02-01 21:15:02

TRIED:  1696  /  288  =  5
2020-02-01 21:20:02

TRIED:  1697  /  288  =  5
2020-02-01 21:25:03

TRIED:  1698  /  288  =  5
2020-02-01 21:30:02

TRIED:  1699  /  288  =  5
2020-02-01 21

TRIED:  1952  /  288  =  6
2020-02-02 18:40:01

TRIED:  1953  /  288  =  6
2020-02-02 18:45:02

TRIED:  1954  /  288  =  6
2020-02-02 18:50:02

TRIED:  1955  /  288  =  6
2020-02-02 18:55:02

TRIED:  1956  /  288  =  6
2020-02-02 19:00:02

TRIED:  1957  /  288  =  6
2020-02-02 19:05:02

TRIED:  1958  /  288  =  6
2020-02-02 19:10:02

TRIED:  1959  /  288  =  6
2020-02-02 19:15:02

TRIED:  1960  /  288  =  6
2020-02-02 19:20:02

TRIED:  1961  /  288  =  6
2020-02-02 19:25:02

TRIED:  1962  /  288  =  6
2020-02-02 19:30:02

TRIED:  1963  /  288  =  6
2020-02-02 19:35:02

TRIED:  1964  /  288  =  6
2020-02-02 19:40:02

TRIED:  1965  /  288  =  6
2020-02-02 19:45:02

TRIED:  1966  /  288  =  6
2020-02-02 19:50:02

TRIED:  1967  /  288  =  6
2020-02-02 19:55:02

TRIED:  1968  /  288  =  6
2020-02-02 20:00:02

TRIED:  1969  /  288  =  6
2020-02-02 20:05:02

TRIED:  1970  /  288  =  6
2020-02-02 20:10:02

TRIED:  1971  /  288  =  6
2020-02-02 20:15:02

TRIED:  1972  /  288  =  6
2020-02-02 20

...

TRIED:  17791  /  288  =  61
2020-03-28 18:35:02

TRIED:  17792  /  288  =  61
2020-03-28 18:40:02

TRIED:  17793  /  288  =  61
2020-03-28 18:45:02

TRIED:  17794  /  288  =  61
2020-03-28 18:50:02

TRIED:  17795  /  288  =  61
2020-03-28 18:55:02

TRIED:  17796  /  288  =  61
2020-03-28 19:00:02

TRIED:  17797  /  288  =  61
2020-03-28 19:05:02

TRIED:  17798  /  288  =  61
2020-03-28 19:10:02

TRIED:  17799  /  288  =  61
2020-03-28 19:15:02

TRIED:  17800  /  288  =  61
2020-03-28 19:20:02

TRIED:  17801  /  288  =  61
2020-03-28 19:25:02

TRIED:  17802  /  288  =  61
2020-03-28 19:30:02

TRIED:  17803  /  288  =  61
2020-03-28 19:35:02

TRIED:  17804  /  288  =  61
2020-03-28 19:40:02

TRIED:  17805  /  288  =  61
2020-03-28 19:45:02

TRIED:  17806  /  288  =  61
2020-03-28 19:50:02

TRIED:  17807  /  288  =  61
2020-03-28 19:55:02

TRIED:  17808  /  288  =  61
2020-03-28 20:00:02

TRIED:  17809  /  288  =  61
2020-03-28 20:05:27

TRIED:  17810  /  288  =  61
2020-03-28 20:10:02



TRIED:  18078  /  288  =  62
2020-03-29 18:30:02

TRIED:  18079  /  288  =  62
2020-03-29 18:35:02

TRIED:  18080  /  288  =  62
2020-03-29 18:40:03

TRIED:  18081  /  288  =  62
2020-03-29 18:45:02

TRIED:  18082  /  288  =  62
2020-03-29 18:50:02

TRIED:  18083  /  288  =  62
2020-03-29 18:55:02

TRIED:  18084  /  288  =  62
2020-03-29 19:00:02

TRIED:  18085  /  288  =  62
2020-03-29 19:05:02

TRIED:  18086  /  288  =  62
2020-03-29 19:10:02

TRIED:  18087  /  288  =  62
2020-03-29 19:15:02

TRIED:  18088  /  288  =  62
2020-03-29 19:20:02

TRIED:  18089  /  288  =  62
2020-03-29 19:25:02

TRIED:  18090  /  288  =  62
2020-03-29 19:30:27

TRIED:  18091  /  288  =  62
2020-03-29 19:35:02

TRIED:  18092  /  288  =  62
2020-03-29 19:40:02

TRIED:  18093  /  288  =  62
2020-03-29 19:45:02

TRIED:  18094  /  288  =  62
2020-03-29 19:50:02

TRIED:  18095  /  288  =  62
2020-03-29 19:55:02

TRIED:  18096  /  288  =  62
2020-03-29 20:00:02

TRIED:  18097  /  288  =  62
2020-03-29 20:05:02



TRIED:  18386  /  288  =  63
2020-03-30 20:10:02

TRIED:  18387  /  288  =  63
2020-03-30 20:15:02

TRIED:  18388  /  288  =  63
2020-03-30 20:20:03

TRIED:  18389  /  288  =  63
2020-03-30 20:25:02

TRIED:  18390  /  288  =  63
2020-03-30 20:30:02

TRIED:  18391  /  288  =  63
2020-03-30 20:35:02

TRIED:  18392  /  288  =  63
2020-03-30 20:40:02

TRIED:  18393  /  288  =  63
2020-03-30 20:45:02

TRIED:  18394  /  288  =  63
2020-03-30 20:50:02

TRIED:  18395  /  288  =  63
2020-03-30 20:55:02

TRIED:  18396  /  288  =  63
2020-03-30 21:00:02

TRIED:  18397  /  288  =  63
2020-03-30 21:05:02

TRIED:  18398  /  288  =  63
2020-03-30 21:10:02

TRIED:  18399  /  288  =  63
2020-03-30 21:15:03

TRIED:  18400  /  288  =  63
2020-03-30 21:20:02

TRIED:  18401  /  288  =  63
2020-03-30 21:25:02

TRIED:  18402  /  288  =  63
2020-03-30 21:30:02

TRIED:  18403  /  288  =  63
2020-03-30 21:35:02

TRIED:  18404  /  288  =  63
2020-03-30 21:40:02

TRIED:  18405  /  288  =  63
2020-03-30 21:45:02



TRIED:  18663  /  288  =  64
2020-03-31 19:15:02

TRIED:  18664  /  288  =  64
2020-03-31 19:20:02

TRIED:  18665  /  288  =  64
2020-03-31 19:25:02

TRIED:  18666  /  288  =  64
2020-03-31 19:30:02

TRIED:  18667  /  288  =  64
2020-03-31 19:35:02

TRIED:  18668  /  288  =  64
2020-03-31 19:40:02

TRIED:  18669  /  288  =  64
2020-03-31 19:45:02

TRIED:  18670  /  288  =  64
2020-03-31 19:50:02

TRIED:  18671  /  288  =  64
2020-03-31 19:55:02

TRIED:  18672  /  288  =  64
2020-03-31 20:00:02

TRIED:  18673  /  288  =  64
2020-03-31 20:05:02

TRIED:  18674  /  288  =  64
2020-03-31 20:10:02

TRIED:  18675  /  288  =  64
2020-03-31 20:15:02

TRIED:  18676  /  288  =  64
2020-03-31 20:20:02

TRIED:  18677  /  288  =  64
2020-03-31 20:25:02

TRIED:  18678  /  288  =  64
2020-03-31 20:30:02

TRIED:  18679  /  288  =  64
2020-03-31 20:35:02

TRIED:  18680  /  288  =  64
2020-03-31 20:40:02

TRIED:  18681  /  288  =  64
2020-03-31 20:45:02

TRIED:  18682  /  288  =  64
2020-03-31 20:50:02



TRIED:  18942  /  288  =  65
2020-04-01 18:30:02

TRIED:  18943  /  288  =  65
2020-04-01 18:35:02

TRIED:  18944  /  288  =  65
2020-04-01 18:40:02

TRIED:  18945  /  288  =  65
2020-04-01 18:45:03

TRIED:  18946  /  288  =  65
2020-04-01 18:50:02

TRIED:  18947  /  288  =  65
2020-04-01 18:55:02

TRIED:  18948  /  288  =  65
2020-04-01 19:00:02

TRIED:  18949  /  288  =  65
2020-04-01 19:05:02

TRIED:  18950  /  288  =  65
2020-04-01 19:10:03

TRIED:  18951  /  288  =  65
2020-04-01 19:15:02

TRIED:  18952  /  288  =  65
2020-04-01 19:20:02

TRIED:  18953  /  288  =  65
2020-04-01 19:25:02

TRIED:  18954  /  288  =  65
2020-04-01 19:30:02

TRIED:  18955  /  288  =  65
2020-04-01 19:35:02

TRIED:  18956  /  288  =  65
2020-04-01 19:40:02

TRIED:  18957  /  288  =  65
2020-04-01 19:45:02

TRIED:  18958  /  288  =  65
2020-04-01 19:50:02

TRIED:  18959  /  288  =  65
2020-04-01 19:55:02

TRIED:  18960  /  288  =  65
2020-04-01 20:00:02

TRIED:  18961  /  288  =  65
2020-04-01 20:05:02



In [3]:

            
for station_i, station in enumerate(stations):
    last_bikes= 0
    last_percent_bikes= 0
    for day_i, data_day in enumerate(station.data_days):
        for val_i, val in enumerate(data_day.bikes):
            if val == EMPTY_DATA_DAY_VAL:
                stations[station_i].data_days[day_i].populate(val_i, day_i * DATAPOINTS_PER_DAY + val_i, last_bikes, last_percent_bikes)
            else:
                last_bikes= data_day.bikes[val_i]
                last_percent_bikes= data_day.percent_bikes[val_i]

In [28]:
# FEATURE DATA PREPERATION

fullness= np.full((TOTAL_TIME_DATAPOINTS, NUM_STATIONS), DUD_VALUE, dtype=np.int)
fullness_in10= np.full((TOTAL_TIME_DATAPOINTS, NUM_STATIONS), DUD_VALUE, dtype=np.int)
fullness_in30= np.full((TOTAL_TIME_DATAPOINTS, NUM_STATIONS), DUD_VALUE, dtype=np.int)
fullness_in60= np.full((TOTAL_TIME_DATAPOINTS, NUM_STATIONS), DUD_VALUE, dtype=np.int)
fullness_percent= np.full((TOTAL_TIME_DATAPOINTS, NUM_STATIONS), DUD_VALUE, dtype=np.float)
bikes_changes_pastx= np.full((TOTAL_TIME_DATAPOINTS, NUM_STATIONS, int(MAX_HINDSIGHT / DATAPOINT_EVERYX_MIN)), DUD_VALUE, dtype=np.int)
day_of_week= np.full((TOTAL_TIME_DATAPOINTS, len(DAYS_OF_WEEK)), DUD_VALUE, dtype=np.int)
hour_of_day= np.full((TOTAL_TIME_DATAPOINTS, HOURS), DUD_VALUE, dtype=np.float)
average_weekday_fullness= np.full((DATAPOINTS_PER_DAY, NUM_STATIONS, len(DAYS_OF_WEEK)), DUD_VALUE, dtype=np.float)
weekdays_vol= np.full((NUM_STATIONS, len(DAYS_OF_WEEK)), 0, dtype=np.float)

station_index_decrement= 0 # this is a varying offset for the indexing of stations that accounts for missing stations that are being ignored
for epoch_day_i in range(TOTAL_DAYS):
    #print("########### epoch_day_i: ", epoch_day_i)
    x_offset= epoch_day_i * DATAPOINTS_PER_DAY
    y_offset= 0
    
    block= np.zeros((DATAPOINTS_PER_DAY, HOURS), dtype=np.float)
    daily_epoch_time= list(range(DATAPOINTS_PER_DAY))
    for time_i in daily_epoch_time:
        hour= float("{:.3f}".format(time_i / 12)) # divide by 12 because there are 12 datapoints in an hour
        block[time_i][(int(hour) + 1) % HOURS]= hour % 1
        block[time_i][int(hour)]= 1 - (hour % 1)
    hour_of_day[x_offset:x_offset + block.shape[0], y_offset:y_offset + block.shape[1]]= block
    
    weekday= stations[2].data_days[epoch_day_i].day_of_week
    block= np.zeros((DATAPOINTS_PER_DAY, len(DAYS_OF_WEEK)), dtype=np.int)
    for block_i, sub_arr in enumerate(block):
        block[block_i][weekday]= 1
    day_of_week[x_offset:x_offset + block.shape[0], y_offset:y_offset + block.shape[1]]= block
    
    for station in stations:
        #print("###### station.index: ", station.index)
        if station.index == 0:
            station_index_decrement= 0
        if station.index in MISSING_STATIONS:
            station_index_decrement+= 1
            continue
        y_offset= station.index - station_index_decrement
        
        block= station.data_days[epoch_day_i].percent_bikes
        block= np.reshape(block, (DATAPOINTS_PER_DAY, 1))
        fullness_percent[x_offset:x_offset + block.shape[0], y_offset:y_offset + block.shape[1]]= block
        
        block= station.data_days[epoch_day_i].bikes
        block= np.reshape(block, (DATAPOINTS_PER_DAY, 1))
        fullness[x_offset:x_offset + block.shape[0], y_offset:y_offset + block.shape[1]]= block
        block= np.reshape(block, (DATAPOINTS_PER_DAY, 1, 1))
        if weekdays_vol[y_offset, weekday] < DAYS_PER_WEEKDAY:
            average_weekday_fullness[0:DATAPOINTS_PER_DAY, y_offset:y_offset + block.shape[1], weekday:weekday+1]+= block
            weekdays_vol[y_offset:y_offset+1, weekday:weekday+1]+= 1
        
        bikes= station.data_days[epoch_day_i].bikes
        block= np.reshape(bikes[2:], (bikes.shape[0] - 2, 1))
        fullness_in10[x_offset:x_offset + block.shape[0], y_offset:y_offset + block.shape[1]]= block
        block= np.reshape(bikes[6:], (bikes.shape[0] - 6, 1))
        fullness_in30[x_offset:x_offset + block.shape[0], y_offset:y_offset + block.shape[1]]= block
        block= np.reshape(bikes[12:], (bikes.shape[0] - 12, 1))
        fullness_in60[x_offset:x_offset + block.shape[0], y_offset:y_offset + block.shape[1]]= block
        
        block= np.reshape(station.data_days[epoch_day_i].bikes, (DATAPOINTS_PER_DAY, 1))
        if epoch_day_i - 1 == -1:
            prev_block= np.zeros((DATAPOINTS_PER_DAY, 1), dtype=np.int)
        else:
            prev_block= np.reshape(station.data_days[epoch_day_i - 1].bikes, (DATAPOINTS_PER_DAY, 1))
        block_xminchange= np.zeros((DATAPOINTS_PER_DAY, int(MAX_HINDSIGHT / DATAPOINT_EVERYX_MIN)), dtype=np.int)
        fullness_xago= np.zeros((DATAPOINTS_PER_DAY, int(MAX_HINDSIGHT / DATAPOINT_EVERYX_MIN)), dtype=np.int)
        for col_i in range(fullness_xago.shape[1]):
            i= col_i + 1
            fullness_xago[i:DATAPOINTS_PER_DAY, col_i:col_i + 1]= block[0:DATAPOINTS_PER_DAY - i, 0:1]
            fullness_xago[0:i, col_i:col_i + 1]= prev_block[DATAPOINTS_PER_DAY - i:DATAPOINTS_PER_DAY, 0:1]
        for col_i in range(fullness_xago.shape[1]):
            block_xminchange[0:DATAPOINTS_PER_DAY, col_i:col_i + 1]= np.subtract(block, fullness_xago[0:DATAPOINTS_PER_DAY, col_i:col_i + 1])
        
        bikes_changes_pastx[x_offset:x_offset + block_xminchange.shape[0], y_offset:y_offset + 1, 0:block_xminchange.shape[1]]= np.reshape(block_xminchange, (DATAPOINTS_PER_DAY, 1, block_xminchange.shape[1]))

station_index_decrement= 0 # this is a varying offset for the indexing of stations that accounts for missing stations that are being ignored
for station in stations:
    if station.index == 0: # [117, 116, 70, 60, 46, 35, 20, 14, 1, 0]
        station_index_decrement= 0
    if station.index in MISSING_STATIONS:
        station_index_decrement+= 1
        continue
    y_offset= station.index - station_index_decrement
    for weekday_i in range(len(DAYS_OF_WEEK)):
        average_weekday_fullness[0:DATAPOINTS_PER_DAY, y_offset:y_offset+1, weekday_i:weekday_i+1]/= weekdays_vol[y_offset:y_offset+1, weekday_i:weekday_i+1]

In [ ]:
print(average_weekday_fullness)

In [36]:
# APPROACH DEFINITIONS

def run_baseline(station_name):
    index= get_station_id(station_name)
    max_train_time= DATAPOINTS_PER_DAY * DAYS_PER_WEEKDAY * len(DAYS_OF_WEEK)
    X_test= fullness[max_train_time:TOTAL_TIME_DATAPOINTS, index:index+1]
    y_test= np.arange(max_train_time, TOTAL_TIME_DATAPOINTS, dtype=np.int)
    
    polynomial_features= PolynomialFeatures(degree= 3)
    models= []
    for weekday_i in range(len(DAYS_OF_WEEK)):
        X_train= np.reshape((average_weekday_fullness[0:DATAPOINTS_PER_DAY, index:index+1, weekday_i:weekday_i+1]), (DATAPOINTS_PER_DAY, 1))
        y_train= np.arange(0, DATAPOINTS_PER_DAY, dtype=np.int)
        poly_X_train= polynomial_features.fit_transform(X_train, y_train)
        regr= linear_model.LinearRegression().fit(poly_X_train, y_train)
        models.append(regr)
    
    y_pred= regr.predict(X_test)
    print("R**2 accuracy: ", r2_score(y_test, y_pred) * 100, " %")
    
def run_approach1(station_name):
    index= get_station_id(station_name)
    
    y= np.full((TOTAL_TIME_DATAPOINTS, 3), 0, dtype=np.int) # change the 3 to a 6 to do both stations at once on the generalised-training form of an approach
    y[0:TOTAL_TIME_DATAPOINTS, 0:1]= np.reshape(fullness_in10[:,index], (TOTAL_TIME_DATAPOINTS, 1))
    y[0:TOTAL_TIME_DATAPOINTS, 1:2]= np.reshape(fullness_in30[:,index], (TOTAL_TIME_DATAPOINTS, 1))
    y[0:TOTAL_TIME_DATAPOINTS, 2:3]= np.reshape(fullness_in60[:,index], (TOTAL_TIME_DATAPOINTS, 1))

    X= np.full((TOTAL_TIME_DATAPOINTS, hour_of_day.shape[1] + day_of_week.shape[1] + 3 \
                + 0 * NUM_STATIONS \
               ), 0, dtype=np.float)
    X[0:TOTAL_TIME_DATAPOINTS, 0:7]= day_of_week
    X[0:TOTAL_TIME_DATAPOINTS, 7:31]= hour_of_day
    X[0:TOTAL_TIME_DATAPOINTS, 31:32]= fullness_percent[0:TOTAL_TIME_DATAPOINTS, index:index + 1]
    X[0:TOTAL_TIME_DATAPOINTS, 32:33]= np.reshape((bikes_changes_pastx[0:TOTAL_TIME_DATAPOINTS, index:index + 1, 0:1]), (TOTAL_TIME_DATAPOINTS, 1)) # past5
    X[0:TOTAL_TIME_DATAPOINTS, 33:34]= np.reshape((bikes_changes_pastx[0:TOTAL_TIME_DATAPOINTS, index:index + 1, 1:2]), (TOTAL_TIME_DATAPOINTS, 1)) # past10
    #X[0:TOTAL_TIME_DATAPOINTS, 34:35]= np.reshape((bikes_changes_pastx[0:TOTAL_TIME_DATAPOINTS, index:index+1, 2:3]), (TOTAL_TIME_DATAPOINTS, 1)) # past15
    #X[0:TOTAL_TIME_DATAPOINTS, 35:36]= np.reshape((bikes_changes_pastx[0:TOTAL_TIME_DATAPOINTS, index:index+1, 3:4]), (TOTAL_TIME_DATAPOINTS, 1)) # past20
    #X[0:TOTAL_TIME_DATAPOINTS, 36:37]= np.reshape((bikes_changes_pastx[0:TOTAL_TIME_DATAPOINTS, index:index+1, 4:5]), (TOTAL_TIME_DATAPOINTS, 1)) # past25
    # X[0:TOTAL_TIME_DATAPOINTS, 139:247]= np.reshape((bikes_changes_pastx[0:TOTAL_TIME_DATAPOINTS, 0:NUM_STATIONS, 0:1]), (TOTAL_TIME_DATAPOINTS, 1)) # past5
    # X[0:TOTAL_TIME_DATAPOINTS, 247:355]= np.reshape((bikes_changes_pastx[0:TOTAL_TIME_DATAPOINTS, 0:NUM_STATIONS, 2:3]), (TOTAL_TIME_DATAPOINTS, 1)) # past15
    # X[0:TOTAL_TIME_DATAPOINTS, 355:463]= np.reshape((bikes_changes_pastx[0:TOTAL_TIME_DATAPOINTS, 0:NUM_STATIONS, 8:9]), (TOTAL_TIME_DATAPOINTS, 1)) # past45

    kf= KFold(n_splits= K)
    kf.get_n_splits(X)
    score_sum= 0.0
    i= 1
    for train_index, test_index in kf.split(X):
        X_train, X_test= X[train_index], X[test_index]
        y_train, y_test= y[train_index], y[test_index]
        regr= MLPRegressor(random_state= 1, max_iter= 1000, alpha=0.001).fit(X_train, y_train)
        y_pred= regr.predict(X_test)
        score_sum+= regr.score(X_test, y_test)
        print("R**2 accuracy of data split", i, ": ", regr.score(X_test, y_test) * 100, " %")
        i+= 1
    print("\nAVERAGE R**2 evaluation: ", (score_sum / K) * 100, " %")
    
def run_approach2(station_name):
    index= get_station_id(station_name)
    
    y= np.full((TOTAL_TIME_DATAPOINTS, 3), 0, dtype=np.int) # change the 3 to a 6 to do both stations at once on the generalised-training form of an approach
    y[0:TOTAL_TIME_DATAPOINTS, 0:1]= np.reshape(fullness_in10[:,index], (TOTAL_TIME_DATAPOINTS, 1))
    y[0:TOTAL_TIME_DATAPOINTS, 1:2]= np.reshape(fullness_in30[:,index], (TOTAL_TIME_DATAPOINTS, 1))
    y[0:TOTAL_TIME_DATAPOINTS, 2:3]= np.reshape(fullness_in60[:,index], (TOTAL_TIME_DATAPOINTS, 1))
    
    X= np.full((TOTAL_TIME_DATAPOINTS, 2 + 2 \
            #* bikes_changes_pastx.shape[1] \ # This line is uncommented when training on all stations
           ), -1, dtype=np.float)
    
    positions= []; t= 0
    while t < 2 * math.pi:
        positions.append((1 - (R * math.cos(t) + R), R * math.sin(t) + R))
        t+= STEP_SIZE
    pos_i= 0
    for time_i in range(TOTAL_TIME_DATAPOINTS):
        X[time_i, 0]= positions[pos_i][0]
        X[time_i, 1]= positions[pos_i][1]
        pos_i= (pos_i + 1) % len(positions)
    
    X[0:TOTAL_TIME_DATAPOINTS, 2:3]= fullness_percent[0:TOTAL_TIME_DATAPOINTS, index:index+1]
    X[0:TOTAL_TIME_DATAPOINTS, 3:4]= np.reshape((bikes_changes_pastx[0:TOTAL_TIME_DATAPOINTS, index:index+1, 0:1]), (TOTAL_TIME_DATAPOINTS, 1)) # past5
    # X[0:TOTAL_TIME_DATAPOINTS, 2:110]= bikes_changes_past5
    # X[0:TOTAL_TIME_DATAPOINTS, 110:218]= bikes_changes_past15
    
    neigh= KNeighborsRegressor(n_neighbors= 30, weights='distance')
    cv_scores= cross_val_score(neigh, X, y, cv=5)
    print(cv_scores) # print each cv score (accuracy) and average them
    print('cv_scores mean:{}'.format(np.mean(cv_scores)))

In [37]:
# DRIVER

run_baseline("PORTOBELLO ROAD")
print("--------------------")
#run_baseline("CUSTOM HOUSE QUAY")

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 1)